## Structured output

只有GPT系列的模型验证通过了测试，deepseek和qwen都失败了，但是这两个模型通过agno的提示词约束，以json_mode模式可以成功运行。（但是这是有几率失败的）

国内的模型在工程化应用的方面，还有距离，一开始是function call，现在是structured output。应用基础还待完善

In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv()

api_key = os.getenv("QWEN_API_KEY")
base_url = os.getenv("QWEN_API_BASE_URL")

In [26]:
deepseek_settings = {
  'api_key' : os.getenv("DEEPSEEK_API_KEY"),
  'base_url' : os.getenv("DEEPSEEK_API_BASE_URL"),
  'id' : 'deepseek-chat'
}

In [43]:
gpt_settings = {
  'api_key' : os.getenv("OPENROUTER_API_KEY"),
  'base_url' : os.getenv("OPENROUTER_BASE_URL"),
  'id' : 'openai/gpt-4.1-nano'
}

In [27]:
llm = init_chat_model(model='qwen-plus-latest',api_key=api_key,base_url=base_url,model_provider='openai')

In [28]:
llm = init_chat_model(model=deepseek_settings['id'],api_key=deepseek_settings['api_key'],base_url=deepseek_settings['base_url'],model_provider='openai')

In [2]:
llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [45]:
llm = init_chat_model(model=gpt_settings['id'],api_key=gpt_settings['api_key'],base_url=gpt_settings['base_url'],model_provider='openai')

In [44]:
from langchain_core.prompts import ChatPromptTemplate
from textwrap import dedent

human_message = dedent("""\
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' json.

Passage:
{input}""")

tagging_prompt = ChatPromptTemplate.from_messages([('user',human_message)])

In [46]:
from pydantic import BaseModel,Field

class Classification(BaseModel):
    sentiment:str = Field(description="The sentiment of the text.")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")

structured_llm = llm.with_structured_output(Classification)

In [47]:
inp = "碟中谍8真的感觉一般!"

prompt = tagging_prompt.invoke({'input': inp})
prompt

ChatPromptValue(messages=[HumanMessage(content="Extract the desired information from the following passage.\n\nOnly extract the properties mentioned in the 'Classification' json.\n\nPassage:\n碟中谍8真的感觉一般!", additional_kwargs={}, response_metadata={})])

In [48]:
response = structured_llm.invoke(prompt)

In [49]:
response

Classification(sentiment='Negative', aggressiveness=6, language='Chinese')